## The objectives are as follows:


- Respond to Business Requirement 2:

    - There is a client who wants to know whether a given image of a cherry leaf is healthy or if it has powdery mildew.



## Inputs


    Following sets:

* Train

* Validation

* Test



## Outputs


- Images distribution plot in train, validation, and test set.

- Increasing data diversity by enhancing images.

- Adjusting label predictions during inference by modifying class indices.

- Machine learning model creation and training.

- The trained model should be preserved (saved).

- The learning curve is plotted to visualize the performance of the model.

- Evaluating the model's performance.

- Making predictions on a randomly selected image file.

In [ ]:
import os
current_dir = os.getcwd()
current_dir



In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import joblib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras

from keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping



In [ ]:
my_data_dir = 'inputs/datasets/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'
my_data_dir

version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
  print('There is a version is already available, please create a new version.')
  pass
else:
  os.makedirs(name=file_path)


In [ ]:
labels = os.listdir(train_path)

print(f"The project labels are: {labels}")

version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/images_shapes.pkl")
image_shape

### Retrieve the counts of images in the train, validation, and test datasets, and generate a plot displaying the distribution of images across these datasets.

In [ ]:
image_counts = []
for folder in ['train', 'validation', 'test']:
    counts = [len(os.listdir(f"{my_data_dir}/{folder}/{label}")) for label in labels]
    image_counts.extend(zip([folder]*len(labels), labels, counts))
    print('\n'.join([f"* {folder} - {label}: {count} images" for label, count in zip(labels, counts)]))

sns.set_style("darkgrid")
plt.figure(figsize=(8, 5))
sns.barplot(data=pd.DataFrame(image_counts, columns=['Set', 'Label', 'Frequency']), x='Set', y='Frequency', hue='Label')
plt.savefig(f'{file_path}/labels_distribution.png', bbox_inches='tight', dpi=150)
plt.show()


### Enhancing the training process, by utilizing the ImageDataGenerator library to employ image data augmentation techniques. This will create a varied set of temporary images within the training dataset, thereby enhancing the training process

In [ ]:

image_augmentor = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
)

training_batch_size = 20



### Visualize Augmented Training Images##

In [ ]:


augmented_train_set = image_augmentor.flow_from_directory(
    train_path,
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=training_batch_size,
    class_mode='binary',
    shuffle=True
)

augmented_train_set.class_indices


augmented_validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=training_batch_size,
    class_mode='binary',
    shuffle=False
)

augmented_validation_set.class_indices



augmented_test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=training_batch_size,
                                                                  class_mode='binary',
                                                                  shuffle=False
                                                                  )

augmented_test_set.class_indices





In [ ]:
for _ in range(5):
    img, label = augmented_train_set.next()
    plt.imshow(img[0])
    plt.show()

In [ ]:

for _ in range(5):
    img, label = augmented_validation_set.next()
    plt.imshow(img[0])
    plt.show()

In [ ]:
for _ in range(5):
    img, label = augmented_test_set.next()
    plt.imshow(img[0])
    plt.show()

In [ ]:
joblib.dump(value=augmented_train_set.class_indices,
            filename=f"{file_path}/train_classes_indices.pkl")

# Creating the model section:

#### - Creating an image classification model using convolutional, pooling, dense, and dropout layers, compiles it, and returns the model. Summary() method is then called to display the model architecture summary.

In [36]:
def create_image_classification_model():
    model = Sequential([
        Conv2D(32, (3, 3), input_shape=image_shape, activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
create_image_classification_model().summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)             

- Training an image classification model using 'augmented_train_set' dataset and validating it using the 'augmented_validation_set' dataset. EarlyStopping callback to stop training if the validation accuracy does not improve for _5 consecutive epochs.

In [52]:
validation_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
model = create_image_classification_model()

model.fit(augmented_train_set,
          epochs=10,
          validation_data=augmented_validation_set,
          callbacks=[validation_stop],
          verbose=1)


Epoch 1/10
148/148 [==============================] - 22s 141ms/step - loss: 1.0271 - accuracy: 0.8081 - val_loss: 0.6850 - val_accuracy: 0.5000
Epoch 2/10
148/148 [==============================] - 20s 137ms/step - loss: 0.1708 - accuracy: 0.9440 - val_loss: 0.6818 - val_accuracy: 0.5000
Epoch 3/10
148/148 [==============================] - 20s 136ms/step - loss: 0.1592 - accuracy: 0.9416 - val_loss: 0.6864 - val_accuracy: 0.5000
Epoch 4/10
148/148 [==============================] - 21s 142ms/step - loss: 0.1388 - accuracy: 0.9470 - val_loss: 0.6877 - val_accuracy: 0.5000
Epoch 5/10
148/148 [==============================] - 19s 130ms/step - loss: 0.0919 - accuracy: 0.9667 - val_loss: 0.6875 - val_accuracy: 0.5000
Epoch 6/10
148/148 [==============================] - 20s 135ms/step - loss: 0.1362 - accuracy: 0.9514 - val_loss: 0.6944 - val_accuracy: 0.5000


In [53]:
model.save('outputs/v1/cherry_disease_model.h5')

# Summary

### To Fix:
- play around with Patience (L53)and epoch (l53)